# 📦 Instalación de dependencias
Es necesario instalar las librerías de Python que usaremos en este proyecto, entre ellas:  
- `lxml` y `beautifulsoup4` → para hacer web scraping y analizar el contenido HTML  
- `requests` → para descargar información desde la web  
- `langchain`, `langchain-community`, `langchain_huggingface` → para el procesamiento de documentos y la implementación del pipeline RAG  
- `huggingface-hub` → para acceder a modelos preentrenados de Hugging Face  
- `faiss-cpu` → para el almacenamiento vectorial y la búsqueda semántica  
- `replicate` → para ejecutar el modelo LLaMA-2 mediante la API de Replicate  
- `pypdf` → para la carga y análisis de documentos PDF  


In [ ]:
pip install lxml requests beautifulsoup4 langchain-community langchain huggingface-hub faiss-cpu replicate pypdf langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 594.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 578.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 717.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

# 📌 Instalación e importación de librerías
En esta celda importamos todas las librerías necesarias para:
- Web scraping (`requests`, `BeautifulSoup`, `RecursiveUrlLoader`)
- Procesamiento de texto (`langchain`, `deep_translator`)
- Creación de embeddings y almacenamiento vectorial (`HuggingFaceEmbeddings`, `FAISS`)
- LLM para Q&A (`Replicate`)

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.vectorstores.base import VectorStoreRetriever
from langchain_community.llms import Replicate
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from deep_translator import GoogleTranslator

# 🔍 Función auxiliar
Creamos una función para verificar si un string contiene números.
Esto puede ser útil para identificar enlaces de artículos que suelen incluir años o IDs numéricos.

In [ ]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

# 🌐 Web scraping
Definimos la URL base y una función para extraer texto con BeautifulSoup.  
Luego, usamos `RecursiveUrlLoader` para recorrer la web y cargar los documentos.

In [ ]:
url = "https://cs.uns.edu.ar/~dcm/home/"

def custom_extractor(html_content):
    soup = BeautifulSoup(html_content, "lxml")
    return soup.get_text()

loader = RecursiveUrlLoader(url=url, extractor=custom_extractor, max_depth=2)

web_docs = loader.load()

# 📄 Descarga y procesamiento de PDFs
Aquí descargamos algunos PDFs de la página, los guardamos localmente, extraemos su texto y les agregamos metadatos (el nombre del archivo de origen).


In [ ]:
pdf_urls = [
    "https://cs.uns.edu.ar/~dcm/downloads/Pautas%20Examen%20Final%20Cursado%202017.pdf",
    "https://cs.uns.edu.ar/~dcm/downloads/Pautas%20Examen%20Final%20Cursado%202016.pdf",
    "https://cs.uns.edu.ar/~dcm/downloads/PautasExamenLibreTdP.pdf"
]
pdf_docs = []

for pdf_url in pdf_urls:
    pdf_filename = pdf_url.split("/")[-1]

    response = requests.get(pdf_url)

    if response.status_code == 200:
        with open(pdf_filename, "wb") as f:
            f.write(response.content)

        pdf_loader = PyPDFLoader(pdf_filename)
        pdf_docs.extend(pdf_loader.load())
    else:
        print(f"Error al descargar el PDF: {pdf_url}")

# 📚 Unificación de documentos
Combinamos los documentos obtenidos del scraping y los PDFs en una sola lista para procesarlos juntos.


In [ ]:
docs = web_docs + pdf_docs

# ✂️ División de texto
Partimos los documentos en fragmentos más pequeños con `RecursiveCharacterTextSplitter`.  
Esto es importante porque los modelos manejan mejor bloques cortos de texto.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=75, chunk_overlap=10)
all_splits = text_splitter.split_documents(docs)

# 🧠 Vectorización con FAISS
Creamos embeddings con `HuggingFaceEmbeddings` y los almacenamos en una base vectorial FAISS.


In [ ]:
model_name="BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = FAISS.from_documents(documents=docs, embedding=hf)
retriever = VectorStoreRetriever(vectorstore=vectorstore, search_kwargs={"k":3})

# 🤖 Configuración del LLM (Replicate)
Se define el modelo LLaMA-2 a través de la API de Replicate.  
**Nota**: es necesario tener configurada la variable de entorno `REPLICATE_API_TOKEN`.


In [ ]:
os.environ["REPLICATE_API_TOKEN"] = env.REPLICATE_API_TOKEN

llm = Replicate(
    model="meta/llama-2-7b-chat",
    model_kwargs={
        "temperature": 0.75,
        "max_length": 500,
        "top_p": 1,
        "language": "es"
)

# 📝 Definición del prompt
Creamos una plantilla de prompt para que el modelo responda siempre en español, de forma breve y precisa.


In [ ]:
template = """Usa el siguiente contexto para responder siempre en español.
Se breve y conciso, máximo tres oraciones y responde solo a la pregunta, no des info adicional.
Es la página de un profesor. Habla en tercera persona.

Si la info es incorrecta o no está clara, menciona que no tienes suficiente info.

{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# 🔗 Cadena de Preguntas y Respuestas (Q&A)
Se construye la cadena `RetrievalQA` con el LLM y el retriever definido.


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

# 🎯 Ejemplo de consulta
Ahora probamos el sistema preguntando por los contenidos de los exámenes.


In [ ]:
query = "¿Cuáles son los contenidos del primer y segundo examen escrito del examen libre de Tecnología de Programación?"
response = qa_chain.invoke(query)
print(response['result'])

 ¡Hola! Según la información proporcionada en la página, los contenidos del primer y segundo examen escrito del examen libre de Tecnología de Programación son los siguientes:

Primer Examen Escrito:

* Ingeniería de Software. Proceso y Ciclo de Vida.
* Reusabilidad y Extendibilidad.
* Confiabilidad.
* Conceptos avanzados de herencia.
* Componentes.
* Discusión de artículos científicos.

Segundo Examen Escrito:

* Diseño reutilizable: patrones de diseño.
* Antipatrones - refactoring.
* Implementación: concurrencia.
* Implementación: frameworks orientados a objetos.
* Implementación: comunicación hombre-máquina.
* Implementación: sistemas con requerimientos especiales.
* Discusión de artículos científicos.

Es importante mencionar que los contenidos pueden variar según la versión del examen, por lo que es recomendable consultar la página personal del profesor para obtener la información más actualizada.
